In [1]:
import pygame
import math,numpy as np

pygame 2.1.2 (SDL 2.0.18, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
display_width = 800
display_height = 600

black = (0,0,0)
white = (255,255,255)
red = (255,0,0)
green = (0,255,0)
blue = (0,0,255)

# Bouncing Balls

In [3]:
pix_per_meter = 100
def to_pix_units(x,y=None):
    if (y is None):
        return x*pix_per_meter
    else:
        return (x*pix_per_meter,display_height - y*pix_per_meter)

def to_phys_units(x,y=None):
    if (y is None):
        return x/pix_per_meter
    else:
        return (x/pix_per_meter,(display_height - y)/pix_per_meter)

In [4]:
class Ball():
    def __init__(self,color,x,y,radius,vx,vy):
        self.color = color
        self.x = x
        self.y = y
        self.radius = radius
        self.vx = vx
        self.vy = vy
        self.bounced = False
    
    def update(self,dt=1/60.,gravity=True):
        if (gravity):
            self.vy -= 9.8*dt
        self.x += self.vx*dt
        self.y += self.vy*dt
        if ((self.y < self.radius) & (self.vy < 0)):
            self.bounce('y')
            if (self.bounced):
                self.vx *= np.sqrt(0.8)
                self.bounced = False
            else:
                self.bounced = True
        else:
            self.bounced = False
        if ((self.x < self.radius) & (self.vx < 0)) | \
            ((self.x > to_phys_units(display_width)-self.radius) & (self.vx > 0)):
            self.bounce('x')
    
    def bounce(self,direction,eloss=True):
        if (self.__dict__['v'+direction] < 0):
            self.__dict__[direction] = self.radius
        else:
            self.__dict__[direction] = to_phys_units(display_width)-self.radius
        self.__dict__['v'+direction] *= -1
        if (eloss):
            if (direction == 'y'):
                self.__dict__['v'+direction] *= np.sqrt(0.8)
            elif (direction == 'x'):
                self.__dict__['v'+direction] *= 0.6
    
    def move(self,x,y):
        self.x = x
        self.y = y
    
    def draw(self,screen):
        pygame.draw.circle(screen,self.color,to_pix_units(self.x,self.y),to_pix_units(self.radius))

def draw_arrow(screen, colour, start, end, size=10):
    pygame.draw.line(screen,colour,start,end,2)
    rotation = math.degrees(math.atan2(start[1]-end[1], end[0]-start[0]))+90
    pygame.draw.polygon(screen, colour, ((end[0]+size*math.sin(math.radians(rotation)), end[1]+size*math.cos(math.radians(rotation))), (end[0]+size*math.sin(math.radians(rotation-120)), end[1]+size*math.cos(math.radians(rotation-120))), (end[0]+size*math.sin(math.radians(rotation+120)), end[1]+size*math.cos(math.radians(rotation+120)))))

In [5]:
# initialize the pygame module
pygame.init()
pygame.display.set_caption("Bouncing Ball")
clock = pygame.time.Clock()
fr = 60

balls = []
velscl = 3.0

# create a surface on screen that has the size of 240 x 180
screen = pygame.display.set_mode((display_width,display_height))

# define a variable to control the main loop
running = True
downpos = None
newball = None

while running:
    for event in pygame.event.get():
        
        randcol = (np.random.randint(256),np.random.randint(256),np.random.randint(256))
        if event.type == pygame.QUIT:
            running = False
        
        # mouse button down = add new ball
        if event.type == pygame.MOUSEBUTTONDOWN:
            downpos = to_phys_units(*pygame.mouse.get_pos())
            newball = Ball(randcol,downpos[0],downpos[1],0.2,0,0)
        
        # mouse button up = set new ball velocity
        if event.type == pygame.MOUSEBUTTONUP:
            if (downpos is not None):
                uppos = to_phys_units(*pygame.mouse.get_pos())
                newball.vx = (uppos[0] - downpos[0])*velscl
                newball.vy = (uppos[1] - downpos[1])*velscl
                balls.append(newball)
                downpos = None
                newball = None
    
    screen.fill(white)
    if (newball is not None):
        newball.draw(screen)
        draw_arrow(screen,randcol,to_pix_units(*downpos),pygame.mouse.get_pos())
    for b in balls:
        b.update()
        b.draw(screen)
    
    pygame.display.update()
    clock.tick(60)

pygame.quit()